In [1]:
import pandas as pd

## Task 1
##### Cho trước 1 folder gồm các file ở định dạng .csv và .xlsx, hãy thực hiện concat tất cả các file .csv thành 1 file duy nhất

<b>Kiến thức:</b> trong python có module <b><i>os</i></b> giúp các bạn làm việc với hệ điều hành như tạo folder, lấy danh sách file, ...
- ```os.path.exists(path)```: kiểm tra directory truyền vào có tồn tại hay không, nếu đã có return True và ngược lại
- ```os.makedirs(path)```: tạo folder theo đường dẫn truyền vào, nếu đã có sẽ báo lỗi do đó nên đi kèm với điều kiện kiểm tra tồn tại hay chưa
- ```os.listdir(path)```: liệt kê danh sách các file, subfolder trong đường dẫn (chỉ ở mức level 1 nếu trong subfolder còn file sẽ không liệt kê được)
- ```os.walk(path)```: tương tự như os.listdir() nhưng liệt kê được đầy đủ tất cả file trong subfolder

#### tạo dữ liệu giả để sử dụng cho task 1 (run 1 lần), các file được tạo sẽ nằm trong folder fake_data trong cùng thư mục chứa file notebook này

In [2]:
import os
import random
from datetime import datetime

colors = ['red', 'green', 'yellow', 'blue', 'cyan', 'pink', 'black', 'white',]

if not os.path.exists('fake_data/'):
    os.makedirs('fake_data/')

for i in range(5):
    random_colors = random.choices(colors, k = 5)
    df = pd.DataFrame(random_colors, columns=['colors'])
    df.to_csv(f"fake_data/{datetime.now().strftime('%d_%M_%Y')}_{i}.csv", index=False)
    df.to_excel(f"fake_data/{datetime.now().strftime('%d_%M_%Y')}_{i}.xlsx", index=False)

In [ ]:
# Y tuong
## duyet cac file dang co trong folder fake_data
### lay ra file co duoi .csv
### doc noi dung cua file
### append noi dung cua file vao mot list
## kết thúc vòng for thì có một list các df chứa nội dung của các file csv
## concat thành 1 file

In [10]:
# Concat toàn bộ list df sau khi kết thúc vòng for
list_of_df = []

for filename in os.listdir('fake_data/'):
    if filename.endswith('csv'):
        df = pd.read_csv(f'fake_data/{filename}')
        list_of_df.append(df)

task1_result = pd.concat(list_of_df, ignore_index=True)
task1_result.to_csv('task1_result.csv', index=False)

In [ ]:
# Hoặc concat df trong quá trình đọc data từ file csv
task1_result = pd.DataFrame()

for filename in os.listdir('fake_data/'):
    if filename.endswith('csv'):
        df = pd.read_csv(f'fake_data/{filename}')
        task1_result = pd.concat([task1_result, df], ignore_index=True)

task1_result.to_csv('results/task1_result.csv', index=False)

## Task 2
Tương tự như task 1 tuy nhiên yêu cầu là tạo 1 file excel kết quả duy nhất gồm nhiều sheet trong đó mỗi sheet là nội dung của mỗi file csv (tên sheet sẽ là tên file csv).

<b>Kiến thức</b>: Để tạo file excel ta có thể sử dụng ```df.to_excel()``` tuy nhiên cách này không thể tạo nhiều sheet trong 1 file được.<br>
Để tạo nhiều sheet ta thực hiện như sau:
```
>>> writer = pd.ExcelWriter('đường dẫn kèm tên file kết quả')  # tạo writer
>>> df1.to_excel(writer, sheet_name='Sheet1')  # sử dụng df1 để tạo sheet1 trong writer
>>> df2.to_excel(writer, sheet_name='Sheet2')  # sử dụng df2 để tạo sheet2 trong writer
>>> writer.close()  # ghi file
```

Bạn có thể sử dụng hàm for với cách này để tạo nhiều sheet trong vòng for, sau khi duyệt và tạo xong sheet cần lưu ý phải ```writer.save()``` để lưu file

In [12]:
writer = pd.ExcelWriter('task2_result.xlsx')

for filename in os.listdir('fake_data/'):  # duyet file trong folder
    if filename.endswith('csv'):  # lay file co duoi csv
        df = pd.read_csv(f'fake_data/{filename}')  # doc noi dung file
        df.to_excel(writer, sheet_name=filename, index=False)  # tao sheet

writer.close()  # ghi file excel

## Task 3
Cho 2 file .csv trong đó:
- transaction.csv là file chứa thông tin các giao dịch của customer
- fraud_customer.csv là file chứa id của các customer được coi là fraud

Hãy lấy ra các giao dịch của các customer không nằm trong danh sách fraud và chia làm 2 sheet với:
- sheet tên big chứa các giao dịch lớn hơn 50000
- sheet tên small chứa các giao dịch nhỏ hơn 50000

Ngoài ra bổ sung thêm 1 sheet thứ 3 với tên là fraud_transactions chứa các giao dịch của customer nằm trong danh sách fraud.

Các sheet cần được lưu trong cùng 1 file excel.

In [13]:
trans_df = pd.read_csv('transactions.csv')

In [14]:
trans_df.head()

,order_id,txn_time_gmt7,service_group_name,service_name,payer_user_id,payer_user_type,payee_user_id,payee_user_type,amount,command_name,customer_id,merchant
0,1.800000e+22,2018-01-02 15:38:17 UTC,Fund-in,Fund-in - ATM,4,agent,6047,customer,95000,Payment,6047.0,NaN
1,1.800000e+22,2018-01-02 22:10:54 UTC,Fund-in,Fund-in - ATM,4,agent,8000,customer,100000,Payment,8000.0,NaN
2,1.800000e+22,2018-01-02 11:13:29 UTC,Fund-in,Fund-in - TPBank,13908,customer,13908,customer,100000,Payment,13908.0,NaN
3,1.800000e+22,2018-01-02 20:18:25 UTC,Fund-in,Fund-in - TPBank,5,customer,5,customer,250000,Payment,5.0,NaN
4,1.800000e+22,2018-01-02 17:04:26 UTC,Fund-in,Fund-in - Vietinbank,13335,customer,13335,customer,10000,Payment,13335.0,NaN


In [15]:
fraud_customer_df = pd.read_csv('fraud_customer.csv')

In [16]:
fraud_customer_df.head()

,customer_id_fraud
0,17060
1,17106
2,17122
3,17005
4,24742


In [17]:
# điều kiện lấy các bản ghi có khách hàng nằm trong trong danh sách fraud_customer
cond = trans_df['customer_id'].isin(fraud_customer_df['customer_id_fraud'])

In [18]:
# lọc các giao dịch của khách hàng không có trong danh sach fraud_customer
not_fraud_tx_df = trans_df.loc[~cond]

# Tạo 2 dataframe có giao dịch lớn hơn 50000 và giao dịch nhỏ hơn 500000 từ not_fraud_transactions
big_not_fraud_tx_df = not_fraud_tx_df.loc[not_fraud_tx_df['amount'] > 50000]
small_not_fraud_tx_df = not_fraud_tx_df.loc[not_fraud_tx_df['amount'] < 50000]

In [21]:
# lọc các giao dịch của khách hàng có trong danh sách fraud_customer
fraud_tx_df = trans_df.loc[cond]

In [22]:
# ghi file
writer = pd.ExcelWriter('task3_result.xlsx')
big_not_fraud_tx_df.to_excel(writer, sheet_name='big', index=False)
small_not_fraud_tx_df.to_excel(writer, sheet_name='small', index=False)
fraud_tx_df.to_excel(writer, sheet_name='fraud_transactions', index=False)
writer.close()

## Task 4
Với file german_credit_test.csv đã cho. Hãy:
- Hãy tính tổng Credit Amount với mỗi nhóm trong Creditability
- Sắp xếp dữ liệu theo giá trị cột Duration.of.Credit..month

In [23]:
german_df = pd.read_csv('german_credit_test.csv')

In [24]:
german_df.head()

,Unnamed: 0,Creditability,Account.Balance,Duration.of.Credit..month.,Payment.Status.of.Previous.Credit,Purpose,Credit.Amount,Value.Savings.Stocks,Length.of.current.employment,Instalment.per.cent,...,Duration.in.Current.address,Most.valuable.available.asset,Age..years.,Concurrent.Credits,Type.of.apartment,No.of.Credits.at.this.Bank,Occupation,No.of.dependents,Telephone,Foreign.Worker
0,2,1,1,9,3,4,2799,1,2,2,...,2,1,36,2,1,2,1,2,1,1
1,4,1,1,12,3,4,2122,1,2,3,...,2,1,39,2,1,2,1,2,1,2
2,10,1,2,24,2,3,3758,3,1,1,...,4,4,23,2,1,1,1,1,1,1
3,11,1,1,11,3,4,3905,1,2,2,...,2,1,36,2,1,2,1,2,1,1
4,13,1,1,6,3,3,1957,1,3,1,...,4,3,31,2,2,1,1,1,1,1


In [14]:
german_df.columns

Index(['Unnamed: 0', 'Creditability', 'Account.Balance',
       'Duration.of.Credit..month.', 'Payment.Status.of.Previous.Credit',
       'Purpose', 'Credit.Amount', 'Value.Savings.Stocks',
       'Length.of.current.employment', 'Instalment.per.cent',
       'Sex...Marital.Status', 'Guarantors', 'Duration.in.Current.address',
       'Most.valuable.available.asset', 'Age..years.', 'Concurrent.Credits',
       'Type.of.apartment', 'No.of.Credits.at.this.Bank', 'Occupation',
       'No.of.dependents', 'Telephone', 'Foreign.Worker'],
      dtype='object')

In [15]:
german_df.groupby('Creditability')[['Credit.Amount']].sum()

,Credit.Amount
Creditability,
0,594827
1,1055373


In [17]:
german_df.sort_values(by='Duration.of.Credit..month.', ascending=True)

,Unnamed: 0,Creditability,Account.Balance,Duration.of.Credit..month.,Payment.Status.of.Previous.Credit,Purpose,Credit.Amount,Value.Savings.Stocks,Length.of.current.employment,Instalment.per.cent,...,Duration.in.Current.address,Most.valuable.available.asset,Age..years.,Concurrent.Credits,Type.of.apartment,No.of.Credits.at.this.Bank,Occupation,No.of.dependents,Telephone,Foreign.Worker
109,239,1,3,4,2,2,601,1,1,1,...,3,1,23,2,1,1,1,2,1,1
262,538,1,3,4,3,4,1455,1,3,2,...,1,1,42,2,2,2,1,2,1,1
258,531,1,3,4,3,3,1503,1,3,2,...,1,1,42,2,2,2,1,2,1,1
122,260,1,3,5,2,4,3448,1,3,1,...,4,1,74,2,2,1,1,1,1,1
249,511,1,3,6,3,4,2080,3,2,1,...,2,3,24,2,2,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,563,0,2,60,2,4,7408,2,1,4,...,2,2,24,2,2,1,1,1,1,1
422,848,0,2,60,1,4,14782,2,4,3,...,4,4,60,1,3,2,1,1,2,1
346,696,1,3,60,3,3,15653,1,3,2,...,4,3,21,2,2,2,1,1,2,1
343,691,1,3,60,2,3,10144,2,3,2,...,4,1,21,2,2,1,1,1,2,1


## Task 5
Với dữ liệu cho trước hãy trả lời các câu hỏi sau :
- độ tuổi trung bình, nhỏ nhất và lớn nhất của mỗi nghề.
- tỷ lệ giới tính nam với mỗi nghề và sắp xếp theo thứ tự giảm dần về tỷ lệ.
- tỷ lệ giới tính nam và giới tính nữ với mỗi nghề.


Hint: .groupby(), .agg(), ...

In [25]:
users = pd.read_table('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user', 
                      sep='|', index_col='user_id', dtype={'age': int})
users.head()

,age,gender,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [26]:
users.groupby('occupation')['age'].agg(['mean', 'min', 'max'])

,mean,min,max
occupation,,,
administrator,38.746835,21,70
artist,31.392857,19,48
doctor,43.571429,28,64
educator,42.010526,23,63
engineer,36.388060,22,70
entertainment,29.222222,15,50
executive,38.718750,22,69
healthcare,41.562500,22,62
homemaker,32.571429,20,50


In [29]:
# đếm nghề cho cả nam và nữ
count_occupation = users.groupby('occupation')['occupation'].count()
count_occupation

occupation
administrator     79
artist            28
doctor             7
educator          95
engineer          67
entertainment     18
executive         32
healthcare        16
homemaker          7
lawyer            12
librarian         51
marketing         26
none               9
other            105
programmer        66
retired           14
salesman          12
scientist         31
student          196
technician        27
writer            45
Name: occupation, dtype: int64

In [30]:
# đếm nghề cho riêng nam
only_male_df = users.loc[users['gender'] == 'M']
only_male_count_df = only_male_df.groupby('occupation')['occupation'].count()
only_male_count_df

occupation
administrator     43
artist            15
doctor             7
educator          69
engineer          65
entertainment     16
executive         29
healthcare         5
homemaker          1
lawyer            10
librarian         22
marketing         16
none               5
other             69
programmer        60
retired           13
salesman           9
scientist         28
student          136
technician        26
writer            26
Name: occupation, dtype: int64

In [41]:
male_perc = only_male_count_df.div(count_occupation).sort_values(ascending=False) * 100
male_perc

occupation
doctor           100.000000
engineer          97.014925
technician        96.296296
retired           92.857143
programmer        90.909091
executive         90.625000
scientist         90.322581
entertainment     88.888889
lawyer            83.333333
salesman          75.000000
educator          72.631579
student           69.387755
other             65.714286
marketing         61.538462
writer            57.777778
none              55.555556
administrator     54.430380
artist            53.571429
librarian         43.137255
healthcare        31.250000
homemaker         14.285714
Name: occupation, dtype: float64

In [35]:
# đếm nghề cho riêng nam và nữ
all_genders_count_df = users.groupby(['occupation', 'gender'])['gender'].count()
all_genders_count_df.head()

occupation     gender
administrator  F         36
               M         43
artist         F         13
               M         15
doctor         M          7
Name: gender, dtype: int64

In [36]:
all_genders_count_df.div(count_occupation)

occupation     gender
administrator  F         0.455696
               M         0.544304
artist         F         0.464286
               M         0.535714
doctor         M         1.000000
educator       F         0.273684
               M         0.726316
engineer       F         0.029851
               M         0.970149
entertainment  F         0.111111
               M         0.888889
executive      F         0.093750
               M         0.906250
healthcare     F         0.687500
               M         0.312500
homemaker      F         0.857143
               M         0.142857
lawyer         F         0.166667
               M         0.833333
librarian      F         0.568627
               M         0.431373
marketing      F         0.384615
               M         0.615385
none           F         0.444444
               M         0.555556
other          F         0.342857
               M         0.657143
programmer     F         0.090909
               M         0